In [185]:
import requests
import pprint
import pandas as pd

pp = pprint.PrettyPrinter(indent=4)


def get_recipes():
    _app_id = '844ee8f7'
    _app_key = '9b846490c7c34c4f33e70564831f232b'

    all_recipes = requests.get('http://api.yummly.com/v1/api/recipes',
                               params={'_app_id':_app_id,
                                       '_app_key':_app_key,
                                       'q':'chicken',
                                       'maxResult':40}).json()
    return all_recipes['matches']

def recipe_ingr_parse(recipe_list):
    return [(recipe['id'], ingredient) for recipe in recipe_list for ingredient in recipe['ingredients']]

def import_ingre_fc():
    #will want this to be done once and stored in db
    fc_indr_raw = pd.read_csv('master_fc_ing.csv')
    fc_indr_raw['ingredient_name'] = [ingredient.replace('_', ' ') for ingredient in fc_indr_raw['ingredient_name']]
    fc_ingr_only['matched'] = fc_ingr_only[['ingredient_name', 'compound_id']].apply(tuple, axis=1)
    ingredients_fc_id = {ingredient: [] for ingredient in fc_ingr_only['ingredient_name']}
    for ingredient, fc_id in list(fc_ingr_only['matched']):
        if ingredient in ingredients_fc_id:
                ingredients_fc_id[ingredient].append(fc_id)
    return ingredients_fc_id


fc_ingr_only_dict = import_ingre_fc()
def fc_return(recipe_id, ingredient):
    # will need to look in db table long term instead of dict.
    return [(recipe_id, fc_id) for fc_id in fc_ingr_only_dict[ingredient]]

def recipe_fc_single_return(recipe_ingr_list):
    recipe_fc_final = []
    for recipe_id, ingredient in recipe_ing:
        if ingredient in fc_ingr_only_dict.keys():
            recipe_fc_final.extend(fc_return(recipe_id, ingredient))
    return recipe_fc_final

def recipe_list_fc_id(recipe_list):
    recipe_fc_id = {recipe['id']: [] for recipe in recipe_list}
    for recipe_id, fc_id in recipe_fc_final:
        if recipe_id in recipe_fc_id:
            recipe_fc_id[recipe_id].append(fc_id)
    return recipe_fc_id

def get_top_recipe_for_user(raw_recipes, user_fc_data):
    recipe_id_fc_list = recipe_list_fc_id(raw_recipes)
    recipe_to_fc_count = {recipe_id:0 for recipe_id in recipe_id_fc_list.keys()}
    
    for recipe_id, flavor_comp_id in recipe_id_fc_list.items():
        for user_flav, user_score in user_fc_data:
            if user_flav in flavor_comp_id:
                recipe_to_fc_count[recipe_id] += user_score

    return max(recipe_to_fc_count, key=recipe_to_fc_count.get)

In [186]:
#flavor_compound_id , score
fake_user_data = [(748, 1), (879, 4), (50, 2), (3,8), (59,1), (200,2)]

In [187]:
import timeit
fc_ingr_only_dict = import_ingre_fc()

start = timeit.default_timer()
#Gets and cleans recipes from Yummly
raw_yum_recipes = get_recipes()

# Breaks down ingredients to (recipe_id, flavor compound)
recipe_ing = recipe_ingr_parse(raw_yum_recipes)
recipe_fc_final = recipe_fc_single_return(recipe_ing)

# Gets top recipe for user
get_top_recipe_for_user(raw_yum_recipes, fake_user_data)

stop = timeit.default_timer()
print(stop - start, ' seconds')

0.4035247209831141  seconds
